# <u>Outils d'Automatisation des Tables SNDS</u>

## Contexte
Dans le cadre du projet complexe de l'IMT Atlantique nommé **"Visualisation des Parcours de Soins"**, nous avons développé un outil permettant de récupérer et traiter les informations des patients présentes dans la base de données SNDS (Système National des Données de Santé).

## Objectifs
L'objectif principal de cet outil est de :
- Récupérer les informations des patients à partir des tables SNDS.
- Extraire des motifs (patterns) pour permettre une visualisation des séquences d'actes médicaux.
- Faciliter l'analyse pour extraire les parcours de soins.

## Tables Utilisées
L'outil permet de récupérer et traiter les tables suivantes :
- **ir_ben_r** : Informations sur les bénéficiaires.
- **er_prs_f** : Prestations.
- **er_pha_f** : Pharmacies.
- **ir_pha_r** : Référentiel des médicaments.
- **er_bio_f** : Actes de biologie.
- **er_cam_f** : Actes de médicaux.
- **er_tip_f** : équipements médicaux.
- **er_ete_f** : Établissements.
- **t_mco23a, t_mco23b, t_mco23c, t_mco23d** : Hospitalisations pour l'année 2023.
- **t_mco22a, t_mco22b, t_mco22c, t_mco22d** : Hospitalisations pour l'année 2022.

## Fonctionnalités
L'outil permet de :
- **Extraire les informations essentielles** pour les économistes de la santé et les analystes de parcours de soins.
- **Automatiser le traitement des données** pour faciliter l'analyse et la visualisation des parcours de soins.
- **Adapter les extractions** selon les besoins spécifiques des utilisateurs.

## Utilisation
Le code est commenté et expliqué pour être le plus adaptable possible aux besoins des utilisateurs. Voici quelques points clés :
- **Commentaires détaillés** : Chaque section du code est commentée pour expliquer son rôle et son fonctionnement.
- **Adaptabilité** : Les utilisateurs peuvent facilement modifier les extractions et les analyses selon leurs besoins spécifiques.


In [ ]:
import pandas as pd
import os
import re
import graphviz

# <u>Chargements des données</u>

Ci-desous se trouve la liste des tables utilisées.

#### **Adaptation des Noms de Fichiers**

En fonction des noms des tables que vous utilisez dans votre base de données, il pourrait être nécessaire d'adapter les noms dans le code.



#### **Exemple d'Adaptation**

Si vos fichiers ont des noms différents de ceux utilisés dans le code, vous devez modifier les noms des variables pour qu'ils correspondent aux noms de vos fichiers.
   Exemple de nom de fichier            | Nom dans le code          |
 |---------------------------|-------------------------------------|
 | `table_des_prestations.xlsx`    |  "table_des_prestations"      |

 Conseil : garder les mêmes noms de tables que dans notre outil pour limiter les changements dans l'ensemble du code

In [ ]:
# Liste des tables à charger
tables_snds = [
    "er_prs_f", "er_pha_f", "ir_pha_r", "er_bio_f", "er_cam_f",
    "er_tip_f", "er_ete_f", "t_mco23b", "t_mco23c", "t_mco23a", "t_mco23d", "t_mco22b", "t_mco22c", "t_mco22a", "t_mco22d","ir_ben_r"
]

Nous travaillons dans le cadre de notre projet avec des tables Excel. Utilisez la fonction suivante :

```python
data_dict[table] = pd.read_excel(file_path, dtype={'BEN_NIR_PSA': str, 'BEN_NIR_ANO': str})
```
Si vous utilisez des csv vous pouvez décommenter la fonction qui se trouve en dessous qui utilise la fonction :
```python
data_dict[table] = pd.read_csv(file_path, dtype={'BEN_NIR_PSA': str, 'BEN_NIR_ANO': str})


In [ ]:
def load_snds_excel(folder_path):
    """
    Charge tous les fichiers .xlsx correspondant aux tables SNDS
    dans un dictionnaire de DataFrames.
    """
    data_dict = {}

    for table in tables_snds:
        file_path = os.path.join(folder_path, f"{table}.xlsx")

        if os.path.exists(file_path):
            print(f"Chargement de {table}...")
            # On charge le fichier.
            # Note : on force le type string pour les identifiants (NIR) pour éviter de perdre le '0' devant.
            data_dict[table] = pd.read_excel(file_path, dtype={'BEN_NIR_PSA': str, 'BEN_NIR_ANO': str}) #Possible ajout de 'NUM_ENQ' : str
        else:
            print(f"⚠️ Attention : Le fichier {table}.xlsx est introuvable dans {folder_path}")

    return data_dict

In [ ]:
"""
def load_snds_csv(folder_path):

    #Charge tous les fichiers .csv correspondant aux tables SNDS
    #dans un dictionnaire de DataFrames.
    data_dict = {}

    for table in tables_snds:
        file_path = os.path.join(folder_path, f"{table}.csv")

        if os.path.exists(file_path):
            print(f"Chargement de {table}...")
            # On charge le fichier CSV.
            # Note : on force le type string pour les identifiants (NIR) pour éviter de perdre le '0' devant.
            data_dict[table] = pd.read_csv(file_path, dtype={'BEN_NIR_PSA': str, 'BEN_NIR_ANO': str}) #Possible ajout de 'NUM_ENQ' : str
        else:
            print(f"⚠️ Attention : Le fichier {table}.csv est introuvable dans {folder_path}")

    return data_dict
    """

'\ndef load_snds_csv(folder_path):\n\n    #Charge tous les fichiers .csv correspondant aux tables SNDS\n    #dans un dictionnaire de DataFrames.\n    data_dict = {}\n\n    for table in tables_snds:\n        file_path = os.path.join(folder_path, f"{table}.csv")\n\n        if os.path.exists(file_path):\n            print(f"Chargement de {table}...")\n            # On charge le fichier CSV.\n            # Note : on force le type string pour les identifiants (NIR) pour éviter de perdre le \'0\' devant.\n            data_dict[table] = pd.read_csv(file_path, dtype={\'BEN_NIR_PSA\': str, \'BEN_NIR_ANO\': str})\n        else:\n            print(f"⚠️ Attention : Le fichier {table}.csv est introuvable dans {folder_path}")\n\n    return data_dict\n    '

Avant de charger les données assurez vous que le chemin est le bon vers vos donnés

In [ ]:
path_to_files = "sample_data" # chemin modifiable vers le dossier contenant les fichiers SNDS
# 1. Charger les données
dfs = load_snds_excel(path_to_files)
#dfs = load_snds_csv(path_to_files) # Décommentez cette ligne pour charger les fichiers CSV

Chargement de er_prs_f...
Chargement de er_pha_f...
Chargement de ir_pha_r...
Chargement de er_bio_f...
Chargement de er_cam_f...
Chargement de er_tip_f...
Chargement de er_ete_f...
Chargement de t_mco23b...
Chargement de t_mco23c...
Chargement de t_mco23a...
Chargement de t_mco23d...
Chargement de t_mco22b...
Chargement de t_mco22c...
Chargement de t_mco22a...
Chargement de t_mco22d...
Chargement de ir_ben_r...


# <u>Filtrage des patients uniques :</u>

Pour notre part, nous avons mis en place certains critères d'exclusion afin d'être sûrs de n'avoir que des patients uniques :  
* Tous les patients ne commençant pas par MR_PARCOURS_SNDS ou NUM_ANO_PAT  
* Tous les BEN_NIR_PSA ayant plusieurs patients rattachés (enfants, jumeaux, …)

Nous avons ensuite effectué une liste avec le BEN_NIR_PSA de ces patients, que nous réutiliserons plus tard.

La suite de notre code se basera principalement sur le BEN_NIR_PSA, car c'est ce qui nous permet d'avoir des patients uniques. **Si vous utilisez le NUM_ENQ, ce code n'est pas approprié.**


In [ ]:
# 2. Filtrer IR_BEN_R pour garantir l'unicité du patient
if 'ir_ben_r' in dfs:
    # On identifie les patients valides
    single_user_psa = (
        dfs['ir_ben_r']
        .groupby('BEN_NIR_PSA')['BEN_NIR_ANO']
        .nunique()
        .loc[lambda x: x == 1]  # Filtre les comptes = 1
        .index
        .tolist()
    )
    single_user_psa.append('MR_PARCOURS_SNDS')
    print(f"Nombre de patients avec un seul compte : {len(single_user_psa)}")
    # On filtre la table de référence
    filtered_df = dfs['ir_ben_r'][
        (dfs['ir_ben_r']['BEN_NIR_PSA'].isin(single_user_psa)) &
        (
        (dfs['ir_ben_r']['BEN_NIR_PSA'].astype(str).str.startswith("NUM_ANO_PAT")) |
        (dfs['ir_ben_r']['BEN_NIR_PSA'].astype(str).str.startswith("MR_PARCOURS_SNDS"))
        )
    ].drop_duplicates('BEN_NIR_PSA')

    # Inclure également les lignes où BEN_NIR_PSA est exactement MR_PARCOURS_SNDS
    mr_parcours_df = dfs['ir_ben_r'][dfs['ir_ben_r']['BEN_NIR_PSA'] == 'MR_PARCOURS_SNDS'].drop_duplicates('BEN_NIR_PSA')

    # Concaténer les deux DataFrames
    filtered_df = pd.concat([filtered_df, mr_parcours_df]).drop_duplicates('BEN_NIR_PSA')

    filtered_user_psa = (
        filtered_df
        .groupby('BEN_NIR_PSA')['BEN_NIR_ANO']
        .nunique()
        .loc[lambda x: x == 1]  # Filtre les comptes = 1
        .index
        .tolist()
    )
    print(f"Vérification après filtrage : Nombre de patients avec un seul compte : {len(filtered_user_psa)}")
else:
    print("Erreur : La table ir_ben_r est indispensable pour le filtrage.")


Nombre de patients avec un seul compte : 998
Vérification après filtrage : Nombre de patients avec un seul compte : 947


# <u>Création d'une table concaténée avec les bonnes colonnes</u>
L'objectif de cette partie est de concaténer les différentes tables ambulatoires (er_prs_f, er_pha_f, ir_pha_r, er_bio_f, er_cam_f, er_tip_f, er_ete_f).

Pour ce faire, nous avons utilisé les clés de jointure suivantes :  
* FLX_EMT_TYP  
* FLX_EMT_NUM  
* REM_TYP_AFF  
* FLX_EMT_ORD  
* FLX_TRT_DTD  
* DCT_ORD_NUM  
* PRS_ORD_NUM  
* FLX_DIS_DTD  
* ORG_CLE_NUM  

Afin de diminuer la taille de er_prs_f, qui est de loin la table la plus volumineuse, nous avons présélectionné les colonnes nécessaires. Si d'autres colonnes vous intéressent, vous pouvez modifier cette partie du code :  
```python
er_prs_f_filtered = er_prs_f_filtered[
    ['BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD',
     'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM',
     'EXE_SOI_DTD', 'EXE_SOI_DTF', 'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT',
     'BSE_REM_PRU', 'CPL_REM_MNT', 'PRS_PAI_MNT', 'BSE_PRS_NAT', 'CPL_PRS_NAT',
     'DPN_QLF', 'PRS_NAT_REF', 'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT', 'PSP_SPE_COD']
]

Il est cependant essentiel de garder les 13 premières :
'BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM',

In [ ]:
# 1. Filtrer er_prs_f avec les colonnes d'identité et les clés de jointure
er_prs_f_filtered = dfs['er_prs_f'][dfs['er_prs_f']['BEN_NIR_PSA'].isin(filtered_user_psa)] #changer les 2 'er_prs_f' si le nom de votre table est différente

# Sélection des colonnes souhaitées, adaptable selon les besoins
er_prs_f_filtered = er_prs_f_filtered[
    ['BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM','EXE_SOI_DTD', 'EXE_SOI_DTF', # ne pas modifier ces lignes
     'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT', 'BSE_REM_PRU', 'CPL_REM_MNT', # peut être modifié
     'PRS_PAI_MNT', 'BSE_PRS_NAT', 'CPL_PRS_NAT', 'DPN_QLF', 'PRS_NAT_REF',     # peut être modifié
     'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT', 'PSP_SPE_COD']                # peut être modifié
]

In [ ]:


# 2. Jointure avec er_pha_f
er_pha_f = dfs['er_pha_f'] #changer le 'er_pha_f' si le nom de votre table est différente
er_pha_f_merged = pd.merge(
    er_prs_f_filtered,
    er_pha_f,
    on=['FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM'],
    how='left'
)

# 3. Jointure avec ir_pha_r
ir_pha_r = dfs['ir_pha_r'] #changer le 'ir_pha_r' si le nom de votre table est différente
final_df = pd.merge(
    er_pha_f_merged,
    ir_pha_r,
    left_on='PHA_PRS_C13',
    right_on='PHA_CIP_C13',
    how='left'
)

# 4. Jointure avec er_bio_f
er_bio_f = dfs['er_bio_f'] #changer le 'er_bio_f' si le nom de votre table est différente
final_df = pd.merge(
    final_df,
    er_bio_f,
    on=['FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM'],
    how='left'
)

# 5. Jointure avec er_cam_f
er_cam_f = dfs['er_cam_f'] #changer le 'er_cam_f' si le nom de votre table est différente
final_df = pd.merge(
    final_df,
    er_cam_f,
    on=['FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM'],
    how='left'
)

# 6. Jointure avec er_tip_f
er_tip_f = dfs['er_tip_f'] #changer le 'er_tip_f' si le nom de votre table est différente
final_df = pd.merge(
    final_df,
    er_tip_f,
    on=['FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM'],
    how='left'
)

# 7. Jointure avec er_ete_f
er_ete_f = dfs['er_ete_f'] #changer le 'er_ete_f' si le nom de votre table est différente
final_df = pd.merge(
    final_df,
    er_ete_f,
    on=['FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM'],
    how='left'
)



## Tableau ambulatoire
Ci-dessous, voici le choix de l'ordre des colonnes du tableau que nous considérons utile pour notre analyse :

Dans er_pha_f :  
'PHA_ACT_PRU', 'PHA_ACT_QSN', 'PHA_CPA_PCP', 'PHA_PRS_C13', 'PHA_SEQ_RNV',

Dans ir_pha_r :  
'PHA_ATC_LIB', 'PHA_ATC_L03',

Dans er_bio_f :  
'BIO_ACT_QSN', 'BIO_PRS_IDE', 'ARO_THE_TAU',

Dans er_cam_f :  
'CAM_ACT_PRU', 'CAM_PRS_IDE',

Dans er_tip_f :  
'TIP_PRS_IDE', 'TIP_ACT_QSN',

Dans er_ete_f :  
'DDP_COD', 'MDT_COD', 'ETE_TYP_COD'

### **Si vous devez changer les colonnes :**

**Étape 1 :**  
Copiez-collez votre sélection de er_prs_f faite précédemment :
```python
'BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD',  
'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM',  
'EXE_SOI_DTD', 'EXE_SOI_DTF'                                                # ne pas modifier ces lignes  
'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT', 'BSE_REM_PRU', 'CPL_REM_MNT',  # peut être modifié  
'PRS_PAI_MNT', 'BSE_PRS_NAT', 'CPL_PRS_NAT', 'DPN_QLF', 'PRS_NAT_REF',      # peut être modifié  
'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT', 'PSP_SPE_COD'                  # peut être modifié  
```
**Étape 2 :**  
Choisissez les colonnes utiles dans les autres tables (ici, tout peut être modifié).

**Étape 3 :**  
Mettez-les dans l'ordre que vous souhaitez.


In [ ]:
# 8. Sélectionner les colonnes utiles dans le bon ordre
final_df = final_df[
    ['BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD',
     'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM',
     'EXE_SOI_DTD', 'EXE_SOI_DTF', 'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT',
     'BSE_REM_PRU', 'CPL_REM_MNT', 'PRS_PAI_MNT', 'BSE_PRS_NAT', 'CPL_PRS_NAT',
     'DPN_QLF', 'PRS_NAT_REF', 'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT', 'PSP_SPE_COD',
     'PHA_ACT_PRU', 'PHA_ACT_QSN', 'PHA_CPA_PCP', 'PHA_PRS_C13', 'PHA_ATC_LIB', 'PHA_ATC_L03', 'PHA_SEQ_RNV',
     'BIO_ACT_QSN', 'BIO_PRS_IDE', 'ARO_THE_TAU', 'CAM_ACT_PRU', 'CAM_PRS_IDE',
     'TIP_PRS_IDE', 'TIP_ACT_QSN', 'DDP_COD', 'MDT_COD', 'ETE_TYP_COD']
]

In [ ]:
print(len(final_df))

9869


**Vous pouvez enregistrer ce tableau sous format Excel en décommentant la case ci-dessous.**

Vous pourrez déjà effectuer une analyse des tables ambulatoires.

In [ ]:
final_df.to_excel("Tableau_ambulatoire.xlsx", index=False)

# <u>Séquences d'actes pour chaque patient, hospitalisations comprises</u>
Dans cette partie, nous allons ajouter les hospitalisations et créer un fichier pour chaque patient, ce qui nous permettra de faire de la visualisation individuelle.

Dans les hospitalisations, nous avons utilisé les tables :  
* t_mcoxxb : NBR_SEA, DGN_PAL, DGN_REL, GRG_GHM, SEJ_NBJ, ENT_MOD, ENT_PRV, SOR_MOD  
* t_mcoxxc : NIR_ANO_17, RNG_NAI, rsa_num, ETA_NUM, HOS_EXE_SOI_DTF, HOS_EXE_SOI_DTD  
* t_mcoxxa : CDC_ACT, NBR_EXE_ACT  
* t_mcoxxd : ASS_DGN  

En utilisant comme clés de jointure :  
* rsa_num  
* ETA_NUM  

**PS : HOS_EXE_SOI_DTF et HOS_EXE_SOI_DTD sont nommées EXE_SOI_DTF et EXE_SOI_DTD dans la table t_mcoxxc, mais ont dû être changées dans le code afin d'éviter les redondances de noms de colonnes.**


Vous pouvez trouver ci-dessous les colonnes que nous avons utilisées dans notre automatisation.  
Si vous souhaitez les modifier, ce sont les mêmes règles que celles vues précédemment.

In [ ]:
# Liste des colonnes pour les prestations ambulatoires
colonnes_ambulatoires = [
    'BEN_NIR_PSA', 'BEN_RNG_GEM', 'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF',                              # ne pas modifier ces lignes
    'FLX_EMT_ORD', 'FLX_TRT_DTD', 'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD',                              # ne pas modifier ces lignes
    'ORG_CLE_NUM', 'EXE_SOI_DTD', 'EXE_SOI_DTF',                                                            # ne pas modifier ces lignes
    'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT', 'BSE_REM_PRU', 'CPL_REM_MNT', 'PRS_PAI_MNT', 'BSE_PRS_NAT',# peut être modifié
    'CPL_PRS_NAT', 'DPN_QLF', 'PRS_NAT_REF', 'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT',                   # peut être modifié
    'PSP_SPE_COD', 'PHA_ACT_PRU', 'PHA_ACT_QSN', 'PHA_CPA_PCP', 'PHA_PRS_C13', 'PHA_ATC_LIB',               # peut être modifié
    'PHA_ATC_L03', 'PHA_SEQ_RNV', 'BIO_ACT_QSN', 'BIO_PRS_IDE', 'ARO_THE_TAU', 'CAM_ACT_PRU',               # peut être modifié
    'CAM_PRS_IDE', 'TIP_PRS_IDE', 'TIP_ACT_QSN', 'DDP_COD', 'MDT_COD', 'ETE_TYP_COD'                        # peut être modifié
]

# Liste des colonnes pour les hospitalisations
colonnes_hospitalisations = [
    'NIR_ANO_17', 'RNG_NAI', 'rsa_num', 'ETA_NUM', 'HOS_EXE_SOI_DTF', 'HOS_EXE_SOI_DTD',    # ne pas modifier ces lignes
    'NBR_SEA', 'DGN_PAL', 'DGN_REL', 'GRG_GHM', 'SEJ_NBJ','ENT_MOD',                        # peut être modifié
    'ENT_PRV', 'SOR_MOD', 'CDC_ACT', 'NBR_EXE_ACT', 'ASS_DGN'                               # peut être modifié
]

In [ ]:
# Fonction pour traiter les hospitalisations pour une année donnée
def process_mco_year(year, tables):
    mco_c = dfs[tables['c']]
    mco_c_filtered = mco_c[mco_c['NIR_ANO_17'].isin(filtered_user_psa)]

    mco_b = dfs[tables['b']]
    mco_b_filtered = mco_b[mco_b['rsa_num'].isin(mco_c_filtered['rsa_num']) &
                           mco_b['ETA_NUM'].isin(mco_c_filtered['ETA_NUM'])]

    mco_a = dfs[tables['a']]
    mco_a_filtered = mco_a[mco_a['rsa_num'].isin(mco_c_filtered['rsa_num']) &
                           mco_a['ETA_NUM'].isin(mco_c_filtered['ETA_NUM'])]

    mco_d = dfs[tables['d']]
    mco_d_filtered = mco_d[mco_d['rsa_num'].isin(mco_c_filtered['rsa_num']) &
                           mco_d['ETA_NUM'].isin(mco_c_filtered['ETA_NUM'])]

    # Jointure des tableaux MCO
    mco_merged = pd.merge(
        mco_c_filtered,
        mco_b_filtered,
        on=['rsa_num', 'ETA_NUM'],
        how='left'
    )

    mco_merged = pd.merge(
        mco_merged,
        mco_a_filtered,
        on=['rsa_num', 'ETA_NUM'],
        how='left'
    )

    mco_merged = pd.merge(
        mco_merged,
        mco_d_filtered,
        on=['rsa_num', 'ETA_NUM'],
        how='left'
    )

    # Ajouter une colonne pour indiquer l'année
    mco_merged['ANNEE'] = year

    # Renommer les colonnes en conflit
    mco_merged = mco_merged.rename(columns={
        'EXE_SOI_DTF': 'HOS_EXE_SOI_DTF',
        'EXE_SOI_DTD': 'HOS_EXE_SOI_DTD'
    })

    return mco_merged[colonnes_hospitalisations]

# Traiter les hospitalisations pour chaque année
hospitalisations_23 = process_mco_year('23', {'a': 't_mco23a', 'b': 't_mco23b', 'c': 't_mco23c', 'd': 't_mco23d'}) #changer les 't_mco23x' si le nom de votre table est différente
hospitalisations_22 = process_mco_year('22', {'a': 't_mco22a', 'b': 't_mco22b', 'c': 't_mco22c', 'd': 't_mco22d'}) #changer les 't_mco23x' si le nom de votre table est différente


In [ ]:
# Concaténer les hospitalisations des deux années
all_hospitalisations = pd.concat([hospitalisations_23, hospitalisations_22], ignore_index=True)

**Vous pouvez enregistrer ce tableau sous format Excel en décommentant la case ci-dessous.**

Vous pourrez déjà effectuer une analyse des tables d'hospitalisations.

In [ ]:
all_hospitalisations.to_excel("Tableau_hospitalisation.xlsx", index=False)

# <u>Tableaux par patient avec l'ensemble des actes </u>

Dans cette partie, nous allons créer un tableau récapitulatif regroupant l’ensemble des actes pour tous les patients, qu’ils soient ambulatoires ou liés à une hospitalisation.
Il est également possible de générer un tableau individuel par patient en décommentant la section dédiée du code.
```python
# Exporter le fichier individuel (optionnel)
    safe_psa = re.sub(r'[\\/*?:"<>|]', '', str(psa))
    file_path = os.path.join("parcours_patients", f"parcours_{safe_psa}.xlsx")
    patient_data.to_excel(file_path, index=False)
    print(f"Parcours pour le patient {psa} exporté dans {file_path}.")
```


Voici une traduction des lignes choisis :    

TYPE → TYPE (Hospitalisation ou Ambulatoire)

BEN_NIR_PSA → Identifiant anonyme du patient dans le SNIIRAM

BEN_RNG_GEM → Rang du bénéficiaire

rsa_num → N° d'index du RSA

ETA_NUM → Numéro FINESS ePMSI

HOS_EXE_SOI_DTD → Date d'exécution des soins hospitaliers

FLX_EMT_TYP → Type de flux émis

FLX_EMT_NUM → Numéro du flux émis

REM_TYP_AFF → Type de remboursement affilié

FLX_EMT_ORD → Ordre du flux émis

FLX_TRT_DTD → Date d'entrée des données dans le système d'information

DCT_ORD_NUM → Numéro d'ordre du décompte dans l'organisme

PRS_ORD_NUM → Numéro d'ordre de la prestation dans le décompte

FLX_DIS_DTD → Date de mise à disposition du flux

ORG_CLE_NUM → Ancien organisme avant fusion (jusqu’au jour J de la fusion)

EXE_SOI_DTD → Date d'exécution des soins

EXE_SOI_DTF → Date de fin d'exécution des soins

PRE_PRE_DTD → Date de prescription

PRS_HOS_DTD → Date de début d'hospitalisation

BSE_REM_MNT → Montant de base du remboursement

BSE_REM_PRU → Prix unitaire de l'acte (acte de base)

CPL_REM_MNT → Montant complémentaire du remboursement

PRS_PAI_MNT → Montant payé par la personne

BSE_PRS_NAT → Nature de base de la personne

CPL_PRS_NAT → Nature complémentaire de la personne

DPN_QLF → Qualificatif de la dépense

PRS_NAT_REF → Nature de la prestation de référence

PSE_ACT_NAT → Nature de l'activité du professionnel de santé

PSE_SPE_COD → Code de spécialité du professionnel de santé

PSP_ACT_NAT → Nature de l'activité du prestataire de santé

PSP_SPE_COD → Code de spécialité du prestataire de santé

PHA_ACT_PRU → Prix unitaire de la prestation affinée de pharmacie

PHA_ACT_QSN → Quantité affinée signée de l'activité pharmaceutique

PHA_CPA_PCP → Condition particulière de prise en charge pharmaceutique

PHA_PRS_C13 → Code prestation affinée (CIP 13)

PHA_ATC_LIB → Libellé ATC (Classification Anatomique Thérapeutique Chimique)

PHA_ATC_L03 → Code ATC niveau 3

PHA_SEQ_RNV → Séquence de renouvellement pharmaceutique

BIO_ACT_QSN → Quantité affinée signée de biologie

BIO_PRS_IDE → Identifiant de la prestation affinée de biologie

ARO_THE_TAU → Taux thérapeutique ARO

CAM_ACT_PRU → Prix unitaire de l'activité CCAM

CAM_PRS_IDE → Identifiant de la prestation affinée CCAM

TIP_PRS_IDE → Identifiant de la prestation affinée TIP

TIP_ACT_QSN → Quantité affinée signée TIP

DDP_COD → Discipline de Prs ou DMT

MDT_COD → Mode de Traitement

ETE_TYP_COD → Type d'établissement de rattachement de l'exécutant ou lieu d'exécution des soins

HOS_EXE_SOI_DTF → Date de fin d'exécution des soins hospitaliers

NBR_SEA → Nombre de séances

DGN_PAL → Diagnostic principal

DGN_REL → Diagnostic relié

GRG_GHM → Groupe Homogène de Malades (GHM)

SEJ_NBJ → Nombre de jours de séjour

ENT_MOD → Mode d'entrée

ENT_PRV → Provenance

SOR_MOD → Mode de sortie

CDC_ACT → Code CCAM (hors extension PMSI)

NBR_EXE_ACT → Nombre d'exécutions d'activité

ASS_DGN → Diagnostic associé

Pour des informations complémentaire sur les lignes : https://health-data-hub.shinyapps.io/dico-snds/

In [ ]:
os.makedirs("parcours_patients", exist_ok=True)  # Vous pouvez modifier le nom du dossier et son emplacement si besoin
# Initialiser une liste pour stocker les DataFrames des patients
all_patients_data = []

 # Liste des colonnes dans l'ordre souhaité


# Itérer sur chaque BEN_NIR_PSA unique
for psa in filtered_user_psa:
    # Filtrer les données pour le BEN_NIR_PSA actuel
    patient_ambulatoire = final_df[final_df['BEN_NIR_PSA'] == psa][colonnes_ambulatoires].copy()
    patient_hospitalisations = all_hospitalisations[all_hospitalisations['NIR_ANO_17'] == psa].copy()

    # Ajouter une colonne pour indiquer le type de données
    patient_ambulatoire['TYPE'] = 'Ambulatoire'
    patient_hospitalisations['TYPE'] = 'Hospitalisation'

    # Renommer NIR_ANO_17 et RNG_NAI pour les hospitalisations
    patient_hospitalisations.rename(columns={'NIR_ANO_17': 'BEN_NIR_PSA', 'RNG_NAI': 'BEN_RNG_GEM'}, inplace=True)

    # Ajouter une colonne de date commune pour le tri chronologique
    patient_ambulatoire['DATE'] = patient_ambulatoire['EXE_SOI_DTD']
    patient_hospitalisations['DATE'] = patient_hospitalisations['HOS_EXE_SOI_DTD']

    # Concaténer les données ambulatoires et hospitalisations
    patient_data = pd.concat([patient_ambulatoire, patient_hospitalisations], ignore_index=True)

    # Trier les actes par ordre chronologique
    patient_data.sort_values(by='DATE', inplace=True)

    # Supprimer la colonne temporaire 'DATE' après le tri
    patient_data.drop(columns=['DATE'], inplace=True)

    colonnes_souhaitees = [
        'TYPE', 'BEN_NIR_PSA', 'BEN_RNG_GEM', 'rsa_num', 'ETA_NUM', 'HOS_EXE_SOI_DTD',
        'FLX_EMT_TYP', 'FLX_EMT_NUM', 'REM_TYP_AFF', 'FLX_EMT_ORD', 'FLX_TRT_DTD',
        'DCT_ORD_NUM', 'PRS_ORD_NUM', 'FLX_DIS_DTD', 'ORG_CLE_NUM', 'EXE_SOI_DTD',
        'EXE_SOI_DTF', 'PRE_PRE_DTD', 'PRS_HOS_DTD', 'BSE_REM_MNT', 'BSE_REM_PRU',
        'CPL_REM_MNT', 'PRS_PAI_MNT', 'BSE_PRS_NAT', 'CPL_PRS_NAT', 'DPN_QLF',
        'PRS_NAT_REF', 'PSE_ACT_NAT', 'PSE_SPE_COD', 'PSP_ACT_NAT', 'PSP_SPE_COD',
        'PHA_ACT_PRU', 'PHA_ACT_QSN', 'PHA_CPA_PCP', 'PHA_PRS_C13', 'PHA_ATC_LIB',
        'PHA_ATC_L03', 'PHA_SEQ_RNV', 'BIO_ACT_QSN', 'BIO_PRS_IDE', 'ARO_THE_TAU',
        'CAM_ACT_PRU', 'CAM_PRS_IDE', 'TIP_PRS_IDE', 'TIP_ACT_QSN', 'DDP_COD',
        'MDT_COD', 'ETE_TYP_COD', 'HOS_EXE_SOI_DTF', 'NBR_SEA', 'DGN_PAL', 'DGN_REL',
        'GRG_GHM', 'SEJ_NBJ', 'ENT_MOD', 'ENT_PRV', 'SOR_MOD', 'CDC_ACT',
        'NBR_EXE_ACT', 'ASS_DGN'
    ]

    # Réorganiser les colonnes du DataFrame patient_data
    patient_data = patient_data[colonnes_souhaitees]

    # Ajouter le DataFrame du patient à la liste
    all_patients_data.append(patient_data)

    # Exporter le fichier individuel (optionnel)
    #safe_psa = re.sub(r'[\\/*?:"<>|]', '', str(psa))
    #file_path = os.path.join("parcours_patients", f"parcours_{safe_psa}.xlsx")
    #patient_data.to_excel(file_path, index=False)
    #print(f"Parcours pour le patient {psa} exporté dans {file_path}.")

# Concaténer tous les DataFrames en un seul
all_patients_concatenated = pd.concat(all_patients_data, ignore_index=True)

# Chemin pour le fichier concaténé
output_file = os.path.join("parcours_patients", "tous_les_parcours_patients.xlsx")

# Exporter le DataFrame concaténé en Excel
all_patients_concatenated.to_excel(output_file, index=False)

print(f"Tous les parcours patients ont été concaténés et exportés dans {output_file}.")


Tous les parcours patients ont été concaténés et exportés dans parcours_patients/tous_les_parcours_patients.xlsx.
